GAIA Clusters <br>
Download the votables by taking the RADIUS = 5. * radius_clusterc <br>
The relative error on parallax and pm are 20% <br>

In [1]:
import sys, os
sys.path.append('../../src')

import astropy.coordinates as coord

from astropy.io.votable import parse
from astropy.table import Table
from astropy import units as u
from astroquery.gaia import Gaia

import numpy as np
import pandas as pd

import gaia_utils as gu

import shutil

%matplotlib inline

## directory
rootdir = "/home/stephane/Science/cluster/GAIA"
wdir    = "%s/products/votable"%(rootdir)
datadir = "%s/master/notebooks/data"%(rootdir)

os.chdir(wdir)

#### Input Parameters########################################################
filelist = datadir+"/"+"BrowseTargets.18292.1530479692.gaia.selected.txt"
fileoutGaia = wdir + "/"+"BrowseTargets.18292.1530479692.gaia.votable.txt"

RADIUS   = 1.0

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [2]:
## read the cluster list from HEASARC
def read_cluster_list(filelist):
    
    df = pd.read_csv(filelist, sep='|')
    
    return(df)

######
def init_SCgaia(filegaia, filelist):
    "init output if not there.."
    
    if not os.path.exists(filegaia):
        with open(filelist,"r") as f:
            header = f.readline()
        with open(filegaia,"w") as f:
            f.write(header)
    
#######
## write in fileoutGaia the selected cluster for GAIA
def write_SCgaia(filegaia, row):
    df = pd.DataFrame(row).T
    df.to_csv(filegaia,sep ="|", mode = "a", header= False, index = False)
    
    
######       
## find the last SC found.
def find_lastSC(filegaia,filelist, dfsc):
    if os.path.exists(filegaia):
        print(filegaia)
        dfcurrent = read_cluster_list(filegaia)
        last_cluster = dfcurrent['name'].iloc[[-1]].iloc[0]
        index = dfsc.index[dfsc['name'].str.contains(last_cluster)]
    else:
        init_SCgaia(filegaia, filelist)
        last_cluster = "No cluster"
        index = [0]
        
    return(last_cluster, index[0])

In [3]:
###############################################################
######  Main loop
###############################################################

df_cluster = read_cluster_list(filelist)
print(df_cluster.index)
print(df_cluster.columns)

lastSC , lastrow = find_lastSC(fileoutGaia, filelist, df_cluster)

print(lastSC)

for index, row in df_cluster.iloc[lastrow:].iterrows():
    clustername = row['name'].strip()
    print("\n\n")
    print("Cluster: %s"%(clustername))
    print("Distance: %3.1f pc"%(row['distance']))
    
    radius = 5. * float(row['cluster_radius'])
    print("Field radius: %3.1f deg"%(radius))
    rasplit = row['ra'].split(' ')
    decsplit = row['dec'].split(' ')
    racluster = "%sh%sm%ss"%(rasplit[0],rasplit[1],rasplit[2])
    deccluster = "%sd%sm"%(decsplit[0],decsplit[1])
    c = coord.SkyCoord(racluster, deccluster, frame='icrs')
    
    cluster = gu.source(clustername)
    
    # filename = get_data(c.ra.deg, c.dec.deg,radius= RADIUS)
    
    filename = cluster.query(radius, coordCluster = [c.ra.deg,c.dec.deg], errtol = 0.2, dump = True)
    
    filedst = "%s-%3.1fdeg.vot"%(clustername, radius)
    shutil.move(filename,filedst)
    
    write_SCgaia(fileoutGaia,row)

RangeIndex(start=0, stop=339, step=1)
Index(['Unnamed: 0', 'name', 'ra', 'dec', 'cluster_radius', 'rad_vel',
       'rad_vel_error', 'num_cluster_stars', 'distance', 'log_age',
       'log_age_error', 'Unnamed: 11'],
      dtype='object')
/home/stephane/Science/cluster/GAIA/products/votable/BrowseTargets.18292.1530479692.gaia.votable.txt
NGC 744          



Cluster: NGC 744
Distance: 1382.0 pc
Field radius: 1.7 deg
SELECT * FROM gaiadr2.gaia_source WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),                                      CIRCLE('ICRS',29.6458333333,55.4783333333,1.7250000000)) = 1  AND abs(pmra_error/pmra)<  0.200000  AND abs(pmdec_error/pmdec)< 0.2000000000 AND abs(parallax_error/parallax)< 0.2000000000;
Launched query: 'SELECT * FROM gaiadr2.gaia_source WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),                                      CIRCLE('ICRS',29.6458333333,55.4783333333,1.7250000000)) = 1  AND abs(pmra_erro

## NGC 6939-1.6deg.vot created
## Query for NGC 6939 done
## Total stars: 42297



Cluster: NGC 189
Distance: 1300.0 pc
Field radius: 0.6 deg
SELECT * FROM gaiadr2.gaia_source WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),                                      CIRCLE('ICRS',9.8708333333,61.1150000000,0.6000000000)) = 1  AND abs(pmra_error/pmra)<  0.200000  AND abs(pmdec_error/pmdec)< 0.2000000000 AND abs(parallax_error/parallax)< 0.2000000000;
Launched query: 'SELECT * FROM gaiadr2.gaia_source WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),                                      CIRCLE('ICRS',9.8708333333,61.1150000000,0.6000000000)) = 1  AND abs(pmra_error/pmra)<  0.200000  AND abs(pmdec_error/pmdec)< 0.2000000000 AND abs(parallax_error/parallax)< 0.2000000000;'
Retrieving async. results...
## NGC 189-0.6deg.vot created
## Query for NGC 189 done
## Total stars: 8472



Cluster: NGC 1502
Distance: 1023.0 pc
Field radius: 1.5 deg
S

## Collinder 463-2.5deg.vot created
## Query for Collinder 463 done
## Total stars: 77242



Cluster: NGC 7708
Distance: 1952.0 pc
Field radius: 1.0 deg
SELECT * FROM gaiadr2.gaia_source WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),                                      CIRCLE('ICRS',354.0750000000,72.8516666667,0.9750000000)) = 1  AND abs(pmra_error/pmra)<  0.200000  AND abs(pmdec_error/pmdec)< 0.2000000000 AND abs(parallax_error/parallax)< 0.2000000000;
Launched query: 'SELECT * FROM gaiadr2.gaia_source WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),                                      CIRCLE('ICRS',354.0750000000,72.8516666667,0.9750000000)) = 1  AND abs(pmra_error/pmra)<  0.200000  AND abs(pmdec_error/pmdec)< 0.2000000000 AND abs(parallax_error/parallax)< 0.2000000000;'
Retrieving async. results...
## NGC 7708-1.0deg.vot created
## Query for NGC 7708 done
## Total stars: 11995



Cluster: MWSC 4674
Distance: 1166.0 pc
Fiel